In [ ]:
def plot_1d_time_series(vector, title="1D Time Series", xlabel="Index", ylabel="Value"):
    """
    Plots a 1D vector as a time series.

    Args:
        vector: A list or numpy array representing the 1D time series.
        title: The title of the plot.
        xlabel: The label for the x-axis.
        ylabel: The label for the y-axis.
    """

    if not isinstance(vector, (list, np.ndarray)):
        raise TypeError("Input 'vector' must be a list or numpy array.")

    if isinstance(vector, list):
        vector = np.array(vector)  # Convert list to numpy array for easier handling

    plt.figure(figsize=(10, 5))  # Adjust figure size as needed
    plt.plot(vector)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)  # Add gridlines for better readability
    plt.show()

In [ ]:
plot_1d_time_series(patients_data[patients_list[0]].loc["mean_SUV"].to_numpy() / max(patients_data[patients_list[0]].loc["mean_SUV"].to_numpy()))
plot_1d_time_series(patients_data[patients_list[10]].loc["mean_SUV"].to_numpy() / max(patients_data[patients_list[10]].loc["mean_SUV"].to_numpy()))
plot_1d_time_series(controls_data[controls_list[10]].loc["mean_SUV"].to_numpy() / max(controls_data[controls_list[10]].loc["mean_SUV"].to_numpy()))

In [ ]:
from sklearn.neighbors import KernelDensity
from scipy.integrate import quad, dblquad

def kde_entropy(kde, dims=1, bounds=(-np.inf, np.inf)):
    """
    Compute entropy H(X) from a kernel density estimate.
    
    Parameters:
    - kde: A trained KernelDensity object.
    - dims: Number of dimensions (1D or 2D).
    - bounds: Integration bounds.
    
    Returns:
    - Entropy value.
    """
    def integrand_1d(x):
        x = np.atleast_1d(x).reshape(-1, 1)  # Ensure x is 1D
        p_x = np.exp(kde.score_samples(x))[0]
        return -p_x * np.log2(p_x) if p_x > 0 else 0

    def integrand_2d(x, y):
        xy = np.array([[x, y]])  # Ensure proper shape
        p_xy = np.exp(kde.score_samples(xy))[0]
        return -p_xy * np.log2(p_xy) if p_xy > 0 else 0

    if dims == 1:
        result, _ = quad(integrand_1d, *bounds)
    else:
        result, _ = dblquad(integrand_2d, *bounds, lambda x: bounds[0], lambda x: bounds[1])
    
    return result

def kde_mutual_information(X, Y, bandwidth=0.1):
    """
    Compute mutual information between two continuous variables using KDE.
    
    Parameters:
    - X, Y: 1D numpy arrays (should be same length or interpolated).
    - bandwidth: KDE bandwidth for smoothing.
    
    Returns:
    - Estimated mutual information value.
    """
    # Train KDE models for each variable
    kde_X = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(X.reshape(-1, 1))
    kde_Y = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(Y.reshape(-1, 1))
    
    # Compute entropy for each variable
    H_X = kde_entropy(kde_X, dims=1)
    H_Y = kde_entropy(kde_Y, dims=1)
    
    # Joint KDE
    min_len = min(len(X), len(Y))
    X_interp = np.interp(np.linspace(0, 1, min_len), np.linspace(0, 1, len(X)), X)
    Y_interp = np.interp(np.linspace(0, 1, min_len), np.linspace(0, 1, len(Y)), Y)
    XY = np.vstack([X_interp, Y_interp]).T
    kde_XY = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(XY)
    
    # Compute joint entropy with 2D integration
    H_XY = kde_entropy(kde_XY, dims=2)

    # Mutual Information: I(X; Y) = H(X) + H(Y) - H(X, Y)
    return H_X + H_Y - H_XY

def compute_mi_matrix(organs, bandwidth=0.1):
    """
    Compute the mutual information matrix for organs of different lengths.
    
    Parameters:
    - organs: Dictionary where keys are organ names and values are 1D numpy arrays (can have different lengths).
    - bandwidth: KDE bandwidth for smoothing.
    
    Returns:
    - A dictionary representing the mutual information matrix.
    """
    n = len(organs)
    mi_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                mi_matrix[i, j] = 0  # Self mutual information is not relevant
            else:
                mi_matrix[i, j] = kde_mutual_information(organs[i], organs[j], bandwidth)
    
    return mi_matrix

organs = patients_data[patients_list[0]].loc["SUV"].to_numpy()
organs.shape
compute_mi_matrix(organs)
